In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
import pandas as pd
df2 = pd.read_csv('McDonald_s_Reviews.csv', encoding="ISO-8859-1")
df2.head()

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star


In [ ]:
def clean(review):
    review = review.lower()
    review = review.replace('ï', '')
    review = review.replace('¿', '')
    review = review.replace('½', '')
    review = review.replace('ý', '')
    review = " ".join(review.split())
    review = review.rstrip()
    return review

df2['clean_review'] = [clean(review) for review in df2['review']]

In [ ]:
#reviews = df2['clean_review']
import re

def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)
df2['clean_review'] = df2['clean_review'].apply(remove_punctuation)


In [ ]:
reviews = df2['clean_review']

In [ ]:
reviews[0]

'why does it look like someone spit on my food i had a normal transaction everyone was chill and polite but now i dont want to eat this im trying not to think about what this milky whiteclear substance is all over my food i d sure am not coming back'

In [ ]:
roberta = "cardiffnlp/twitter-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(roberta)
model.to("cuda")
tokenizer = AutoTokenizer.from_pretrained(roberta,model_max_length = 514)

labels = ['Negative', 'Neutral', 'Positive']

In [ ]:
from tqdm import tqdm
res= []
i = 0
for review in tqdm(reviews):
  try:
    encodedReview = tokenizer(review, return_tensors='pt').to('cuda')
    output = model(**encodedReview)
    scores = output[0][0].detach().cpu().numpy()
    scores = softmax(scores)
    res.append(scores)
  except Exception as e:
    print(e)
    i +=1
    print(i)
    res.append(None)

  # for i in range(len(scores)):
  #   l = labels[i]
  #   s = scores[i]
  #   print(l,round((s*100),2))

 52%|█████▏    | 17325/33396 [05:12<04:36, 58.02it/s]

The expanded size of the tensor (566) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 566].  Tensor sizes: [1, 514]
1


 69%|██████▉   | 23028/33396 [06:51<03:38, 47.54it/s]

The expanded size of the tensor (617) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 617].  Tensor sizes: [1, 514]
2


 69%|██████▉   | 23165/33396 [06:54<03:57, 43.06it/s]

...

 84%|████████▍ | 28103/33396 [08:21<01:30, 58.40it/s]

The expanded size of the tensor (523) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 523].  Tensor sizes: [1, 514]
6


 86%|████████▋ | 28812/33396 [08:34<01:15, 60.76it/s]

The expanded size of the tensor (568) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 568].  Tensor sizes: [1, 514]
7


100%|██████████| 33396/33396 [09:53<00:00, 56.30it/s]


In [ ]:
res[0]

array([0.9752794 , 0.02260427, 0.00211631], dtype=float32)

In [ ]:
import pandas as pd
ress = pd.Series(res)

In [ ]:
ress.to_csv('senimentsBertAllThree.csv')

In [ ]:
ress

0         [0.9752794, 0.022604272, 0.002116306]
1        [0.002685675, 0.023319323, 0.97399503]
2          [0.7752055, 0.21287917, 0.011915322]
3         [0.005768352, 0.09304767, 0.90118396]
4          [0.92126226, 0.07193809, 0.00679967]
                          ...                  
33391    [0.9745224, 0.023127828, 0.0023498212]
33392     [0.0021946663, 0.01530327, 0.9825021]
33393       [0.2315547, 0.69735193, 0.07109343]
33394     [0.58317536, 0.33397937, 0.082845315]
33395      [0.005174789, 0.10072993, 0.8940953]
Length: 33396, dtype: object